https://sdsawtelle.github.io/blog/output/automated-gauge-readout-with-opencv.html

https://stackoverflow.com/questions/46513323/how-to-read-utility-meter-needle-with-opencv

In [1]:
%matplotlib inline
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os

In [2]:
# D:\StudyTime\ProgrammingWorld\CodeZone\Development\AI_ML\Python\PressureGaugeReader\Images\out2\ExtractedImages
# ExtractedImages_Dir = "../Python/PressureGaugeReader/Images/out2/ExtractedImages/"
# imgname = "out2_00000.jpg"
# ExtractedImages_CirularImg_Dir = "../Python/PressureGaugeReader/Images/out2/ExtractedImages_CircularDial/"

imgname = "0.0/out2_00000.jpg"
Train_Dir = "../Python/PressureGaugeReader/Images/out2/Train/"
Train_CirularImg_Dir = "../Python/PressureGaugeReader/Images/out2/Train_CircularDial/"

Test_Dir = "../Python/PressureGaugeReader/Images/out2/Test/"
Test_CirularImg_Dir = "../Python/PressureGaugeReader/Images/out2/Test_CircularDial/"

Validate_Dir = "../Python/PressureGaugeReader/Images/out2/Validate/"
Validate_CirularImg_Dir = "../Python/PressureGaugeReader/Images/out2/Validate_CircularDial/"

In [3]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [4]:
def ReadImage(img_path):
    return cv2.imread(img_path)

In [5]:
img = ReadImage(ExtractedImages_Dir + imgname)
plt.imshow(img)

NameError: name 'ExtractedImages_Dir' is not defined

In [6]:
def cropImage(img, x,y, h,w):
    return img[y:y+h, x:x+w]


In [7]:
x = 70
y = 120

w = 420
h = 420

crop_img = cropImage(img, x, y, h, w)
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB))

NameError: name 'img' is not defined

In [8]:
b,g,r = cv2.split(crop_img)
plt.imshow(r)

NameError: name 'crop_img' is not defined

In [9]:
def Process_Cropping(src, dest, x, y, h, w):
    img = ReadImage(src)
    # plt.imshow(img)
    # break
    image = cropImage(img, x, y, h, w)
    roi = np.zeros(image.shape[:2], np.uint8)
    roi = cv2.circle(roi, (210, 210), 210, 255, cv2.FILLED)

    mask = np.ones_like(image) * 255
    cir_dial = cv2.bitwise_and(mask, image, mask=roi) + cv2.bitwise_and(mask, mask, mask=~roi)
    cv2.imwrite(dest, cir_dial)

In [10]:
x = 70
y = 120

w = 420
h = 420

for i in os.listdir(ExtractedImages_Dir):
    img = ReadImage(ExtractedImages_Dir + i)
    # plt.imshow(img)
    # break
    image = cropImage(img, x, y, h, w)
    roi = np.zeros(image.shape[:2], np.uint8)
    roi = cv2.circle(roi, (210, 210), 210, 255, cv2.FILLED)

    mask = np.ones_like(image) * 255
    cir_dial = cv2.bitwise_and(mask, image, mask=roi) + cv2.bitwise_and(mask, mask, mask=~roi)
    cv2.imwrite(ExtractedImages_CirularImg_Dir +i, cir_dial)

NameError: name 'ExtractedImages_Dir' is not defined

In [11]:
def Process_Dir_Cropping(targetSrcDir, targetDestDir):
    createFolder(targetDestDir)

    for subdir, dirs, files in os.walk(targetSrcDir):
        for dir in dirs:
            createFolder(targetDestDir + dir)
            for file in os.listdir(targetSrcDir + dir):
                src = os.path.join(targetSrcDir + dir, file)
                dest = os.path.join(targetDestDir + dir, file)
                x = 70
                y = 120

                h = 420
                w = 420

                Process_Cropping(src, dest, x, y, h, w)

                    # for file in files:
    #     print(os.path.join(subdir, file))
    #     print(os.path.join(Train_CirularImg_Dir + dir, file))

In [12]:
Process_Dir_Cropping(Train_Dir, Train_CirularImg_Dir)
Process_Dir_Cropping(Test_Dir, Test_CirularImg_Dir)
Process_Dir_Cropping(Validate_Dir, Validate_CirularImg_Dir)          


In [ ]:
img_hsv = cv2.cvtColor(cir_dial,cv2.COLOR_BGR2HSV)  # Convert to HSV space
plt.imshow(img_hsv)

In [ ]:
# Black HSV mask
# lower_black, upper_black = np.array([0, 0, 0]), np.array([180, 255, 65])
# mask = cv2.inRange(img_hsv, lower_black, upper_black)
# fig, ax = plt.subplots(figsize=(7, 7))
# ax.set_title("Black Color Thresholded Mask", fontsize=16)
# ax.imshow(mask, cmap="Greys")

for i in range(100, 255, 50):
    lower_black, upper_black = np.array([0, 0, 0]), np.array([i, 255, 65])
    mask = cv2.inRange(img_hsv, lower_black, upper_black)
    fig, ax = plt.subplots(figsize=(7, 7))
    ax.set_title("Black Color Thresholded Mask - " + str(i), fontsize=16)
    ax.imshow(mask, cmap="Greys")

In [ ]:
invert_img = cv2.bitwise_not(mask)

In [ ]:
# Apply a Gaussian Blur to the inverted mask
blurred = cv2.blur(~invert_img, (9, 9))
fig, ax = plt.subplots(figsize=(7, 7))
ax.set_title("Blurred and Inverted Mask", fontsize=16)
ax.imshow(blurred)

In [ ]:
# Setup SimpleBlobDetector parameters.
params = cv2.SimpleBlobDetector_Params()
 
# Adjust parameters for determining blob stability/repeatability
params.minThreshold = 0
params.maxThreshold = 100
params.minRepeatability = 1
 
# Filter by Area.
params.filterByArea = True
params.minArea = 0 # 0.75 * 118 * rect_area / 8079  # rect_area is found above during red rectangle detection
 
# Filter by Circularity
params.filterByCircularity = True
params.maxCircularity = 5

# Create a detector with the parameters
detector = cv2.SimpleBlobDetector_create(params)

In [ ]:
# # create the small border around the image, just bottom
# blurred2=cv2.copyMakeBorder(blurred, top=0, bottom=1, left=0, right=0, borderType= cv2.BORDER_CONSTANT, value=[255,255,255] ) 

# # create the params and deactivate the 3 filters
# params = cv2.SimpleBlobDetector_Params()
# params.filterByArea = False
# params.filterByInertia = False
# params.filterByConvexity = False

# # detect the blobs
# detector = cv2.SimpleBlobDetector_create(params)
# keypoints = detector.detect(blurred2)

# # display them
# img_with_keypoints = cv2.drawKeypoints(blurred2, keypoints, outImage=np.array([]), color=(0, 0, 255),flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# print(keypoints)
# # cv2.imshow("Frame", img_with_keypoints)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()

In [ ]:
# Detect blobs
keypoints = detector.detect(blurred)
print(keypoints)

# Draw detected blobs as green circles on the blurred mask
imcopy = image.copy()
im_with_keypoints = cv2.drawKeypoints(blurred, keypoints, 
                                      np.array([]), (0,255,0), 
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  # ensures circle size corresponds to blob size
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title("Detected Blobs in Blurred Mask", fontsize=16)
ax.imshow(im_with_keypoints)

In [ ]:
lower_black, upper_black = np.array([0, 0, 0]), np.array([180, 255, 65])
mask = cv2.inRange(img_hsv, lower_black, upper_black)

invert_img = cv2.bitwise_not(mask)

# Apply a Gaussian Blur to the inverted mask
blurred = cv2.blur(~invert_img, (9, 9))

# Detect blobs
keypoints = detector.detect(blurred)

# Draw detected blobs as green circles on the original image
imcopy = image.copy()
im_with_keypoints = cv2.drawKeypoints(imcopy, keypoints, 
                                      np.array([]), (0,255,0), 
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  # ensures circle size corresponds to blob size
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title("Detected Blobs in Blurred Mask", fontsize=16)
ax.imshow(im_with_keypoints)

In [ ]:
# Extract coordinates of gauge centers in the (column index, row index) convention
gauge_center_coords = []
for kp in keypoints:
    gauge_center_coords.append((np.int(kp.pt[0]), np.int(kp.pt[1])))
    
# Sort the gauge's by center coordinates
gauge_center_coords.sort(key=lambda x: x[1])
gauge_center_coords[0:4] = sorted(gauge_center_coords[0:4], key=lambda x: x[0])
gauge_center_coords[4:] = sorted(gauge_center_coords[4:], key=lambda x: x[0])
print("Gauge centers found at coordinates: ", gauge_center_coords)

In [ ]:
def find_gauge_centers(img_hsv, rect_area):
    
    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()
    # Adjust parameters for determining blob stability/repeatability
    params.minThreshold = 0
    params.maxThreshold = 100
    params.minRepeatability = 1
    # Filter by Area.
    params.filterByArea = True
    params.minArea = 0.75 * 118 * rect_area / 8079  # rect_area is found above during red rectangle detection
    # Filter by Circularity
    params.filterByCircularity = True
    params.maxCircularity = 5
    detector = cv2.SimpleBlobDetector_create(params)
    
    # Increment upper bound for "value" (brightness) until we find 6 blobs
    n_blobs = 0
    v_upper = 25
    while n_blobs < 1:
        # Black HSV mask
        lower_black, upper_black = np.array([0, 0, 0]), np.array([180, 255, v_upper])
        mask = cv2.inRange(img_hsv, lower_black, upper_black)
        invert_img = cv2.bitwise_not(mask)
        # Apply a Gaussian Blur to the inverted mask
        blurred = cv2.blur(~invert_img, (9, 9))
        # Detect blobs
        keypoints = detector.detect(blurred)
        n_blobs = len(keypoints)
        v_upper += 5
    
    # Extract the coordinates of gauge centers in the (column index, row index) convention
    gauge_center_coords = []
    for kp in keypoints:
        gauge_center_coords.append((np.int(kp.pt[0]), np.int(kp.pt[1])))
    
    # Sort the gauge's by center coordinates
    gauge_center_coords.sort(key=lambda x: x[1])
    gauge_center_coords[0:4] = sorted(gauge_center_coords[0:4], key=lambda x: x[0])
    gauge_center_coords[4:] = sorted(gauge_center_coords[4:], key=lambda x: x[0])
    print("Gauge centers found at coordinates: ", gauge_center_coords)
    
    return keypoints, gauge_center_coords

In [ ]:
image = cv2.imread(imgname)
img_hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV)  # Convert to HSV space

In [ ]:
# Pick out the first gauge to work on
circ_col, circ_row = gauge_center_coords[0]

In [ ]:
# # Extract rotated rectangle parameters
# rotrect_angle, rect_width, rect_area = rect[2], min(rect[1][0], rect[1][1]), rect[1][0]*rect[1][1]
# rect_col, rect_row = rect[0] 

# Determine good size for circles to be drawn around gauges
circ_rad = np.int(30 * 200 / 57)

# Draw a circle centered around gauge center point, extract pixel indices and colors on circle perimeter
blank = np.zeros(image.shape[:2], dtype=np.uint8)
cv2.circle(blank, (circ_col, circ_row), circ_rad, 255, thickness=1)  # Draw function wants center point in (col, row) order like coordinates
ind_row, ind_col = np.nonzero(blank)
b = image[:, :, 0][ind_row, ind_col]
g = image[:, :, 1][ind_row, ind_col]
r = image[:, :, 2][ind_row, ind_col]
colors = list(zip(b, g, r))

In [ ]:
# "reverse" the row indices to get a right-handed frame of reference with origin in bottom left of image
ind_row_rev = [image.shape[0] - row for row in ind_row]
circ_row_rev = image.shape[0] - circ_row
# rect_row_rev = image.shape[0] - rect_row

# Convert from indexes in (row, col) order to coordinates in (col, row) order
circ_x, circ_y = circ_col, circ_row_rev
original_coord = list(zip(ind_col, ind_row_rev))
# rect_x, rect_y = rect_col, rect_row_rev

# Rotate coords about rectangle center in order to identify topmost pixel of gauges
temp_x, temp_y = [x - rect_x for x in ind_col], [y - rect_y for y in ind_row_rev]  # Translate from rectangle center point
angle_deg = 20 #get_real_angle(rotrect_angle)
theta = angle_deg * (np.pi/180)
rotated = []
for (x, y) in list(zip(temp_x, temp_y)):
    rotated.append(((x*np.cos(theta) - y*np.sin(theta)) + 0, 
                    (y*np.cos(theta) + x*np.sin(theta)) + 0))  # Rotate about 0,0 then reverse translation from rectangle center point
top_yval = max([y for (x,y) in rotated])
top_pixel = [(x, y) for (x, y) in rotated if y == top_yval][0]

# Translate coords from gauge centers in order to compute angle between points on the perimeter
translated = []
for (x, y) in original_coord:
    translated.append((x - circ_x, y - circ_y))

In [ ]:
# Construct dataframe holding various coordinate representations and pixel values
df = pd.DataFrame({"indices":list(zip(ind_col, ind_row)), "orig":original_coord, "rot": rotated, "trans": translated, "color": colors})

# Identify the pixel which is the topmost point of the circle when properly rotated
df["top_pixel"] = (df["rot"] == top_pixel)
top_trans_pix = df.loc[df["top_pixel"], "trans"].values[0]
df.head()

In [ ]:
# Visualize the circle and topmost circle pixel
imcopy = image.copy()
fig, ax = plt.subplots(figsize=(18, 6))
ax.set_title("Sample Perimeter Circle and Topmost Pixel", fontsize=16)
cv2.circle(imcopy, (circ_col, circ_row), circ_rad, 255, thickness=1)  # Draw circle around gauge center point
top_orig_pix =  df.loc[df["top_pixel"], "indices"].values[0]  # Get indices for "topmost" pixel on circle after rotation
cv2.circle(imcopy, top_orig_pix, 1, 255, thickness=3)  # Draw topmost pixel
ax.imshow(imcopy)

Locating Gauge Needles

In [ ]:
# Determine good size for circles to be drawn around gauges
circ_rad = np.int(30 * rect_width / 57)

# Draw a circle centered around gauge center point, extract pixel indices and colors on circle perimeter
blank = np.zeros(image.shape[:2], dtype=np.uint8)
cv2.circle(blank, (circ_col, circ_row), circ_rad, 255, thickness=1)  # Draw function wants center point in (col, row) order like coordinates
ind_row, ind_col = np.nonzero(blank)
b = image[:, :, 0][ind_row, ind_col]
g = image[:, :, 1][ind_row, ind_col]
r = image[:, :, 2][ind_row, ind_col]
colors = list(zip(b, g, r))

# "reverse" the row indices to get a right-handed frame of reference with origin in bottom left of image
ind_row_rev = [image.shape[0] - row for row in ind_row]
circ_row_rev = image.shape[0] - circ_row
rect_row_rev = image.shape[0] - rect_row

# Convert from indexes in (row, col) order to coordinates in (col, row) order
circ_x, circ_y = circ_col, circ_row_rev
original_coord = list(zip(ind_col, ind_row_rev))
rect_x, rect_y = rect_col, rect_row_rev

# Rotate coords about rectangle center in order to identify topmost pixel of gauges
temp_x, temp_y = [x - rect_x for x in ind_col], [y - rect_y for y in ind_row_rev]  # Translate from rectangle center point
angle_deg = get_real_angle(rotrect_angle)
theta = angle_deg * (np.pi/180)
rotated = []
for (x, y) in list(zip(temp_x, temp_y)):
    rotated.append(((x*np.cos(theta) - y*np.sin(theta)) + rect_x, 
                    (y*np.cos(theta) + x*np.sin(theta)) + rect_y))  # Rotate about 0,0 then reverse translation from rectangle center point
top_yval = max([y for (x,y) in rotated])
top_pixel = [(x, y) for (x, y) in rotated if y == top_yval][0]

# Translate coords from gauge centers in order to compute angle between points on the perimeter
translated = []
for (x, y) in original_coord:
    translated.append((x - circ_x, y - circ_y))

# Construct dataframe holding various coordinate representations and pixel values
df = pd.DataFrame({"indices":list(zip(ind_col, ind_row)), "orig":original_coord, "rot": rotated, "trans": translated, "color": colors})

# Identify the pixel which is the topmost point of the circle when properly rotated
df["top_pixel"] = (df["rot"] == top_pixel)
top_trans_pix = df.loc[df["top_pixel"], "trans"].values[0]
df.head()

# Visualize the circle and topmost circle pixel
imcopy = image.copy()
fig, ax = plt.subplots(figsize=(18, 6))
ax.set_title("Sample Perimeter Circle and Topmost Pixel", fontsize=16)
cv2.circle(imcopy, (circ_col, circ_row), circ_rad, 255, thickness=1)  # Draw circle around gauge center point
top_orig_pix =  df.loc[df["top_pixel"], "indices"].values[0]  # Get indices for "topmost" pixel on circle after rotation
cv2.circle(imcopy, top_orig_pix, 1, 255, thickness=3)  # Draw topmost pixel
ax.imshow(imcopy)

In [ ]:
import tkinter
from tkinter import *
import tkinter as tk
import tkinter.messagebox as mbox
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw
import numpy as np
import cv2

In [ ]:
# Main Window
frame = Tk()
frame.title('Circular Image Cropper')
frame.geometry('1300x750')
# frame.configure(bg = "white")

# image on the main window
path = "image2.jpg"
# Creates a Tkinter-compatible photo image, which can be used everywhere Tkinter expects an image object.
img1 = ImageTk.PhotoImage(Image.open(path))
# The Label widget is a standard Tkinter widget used to display a text or image on the screen.
panel = tk.Label(frame, image = img1)
panel.place(x = 270, y = 150)

# starting label
start1 = Label(frame, text='CIRCULAR IMAGE CROPPER', font=("Arial", 55,"underline"),fg="magenta")
start1.place(x=120,y=10)

def start_fun():
    frame.destroy()

# creating an exit button
prevB = Button(frame, text='START', command=start_fun, font=("Arial", 25), bg = "light green", fg = "blue", borderwidth=3, relief="raised")
prevB.place(x = 200, y = 620)

# defined exit_win function, to show a exit dialog box when tried to exit
def exit_win():
    if mbox.askokcancel("Exit", "Do you want to exit?"):
        frame.destroy()

# creating an exit button
prevB = Button(frame, text='EXIT', command=exit_win, font=("Arial", 25), bg = "red", fg = "blue", borderwidth=3, relief="raised")
prevB.place(x = 970, y = 620)

# this is done to show the exit dialog box when tried to exit from the main window, using the top-roght close button of titlebar
frame.protocol("WM_DELETE_WINDOW", exit_win)
frame.mainloop()

#created main window
window = Tk()
window.geometry("1300x750")
window.title("Circular Image Cropper")

# defined variable
global count, emig
# global bright, con
# global frp, tname  # list of paths
frp = []
tname = []
con = 1
bright = 0
panelB = None
panelA = None



In [ ]:
# function defined to get the path of the image selected
def getpath(path):
    a = path.split(r'/')
    # print(a)
    fname = a[-1]
    l = len(fname)
    location = path[:-l]
    return location

# function defined to get the folder name from which image is selected
def getfoldername(path):
    a = path.split(r'/')
    # print(a)
    name = a[-1]
    return name

# function defined to get the file name of image is selected
def getfilename(path):
    a = path.split(r'/')
    fname = a[-1]
    a = fname.split('.')
    a = a[0]
    return a

# function defined to open the image file
def openfilename():
    filename = filedialog.askopenfilename(title='"pen')
    return filename

# function defined to open the selected image
def open_img():
    global x, panelA, panelB
    global count, eimg, location, filename
    count = 0
    x = openfilename()
    img = Image.open(x)
    eimg = img
    img = ImageTk.PhotoImage(img)
    temp = x
    location = getpath(temp)
    filename = getfilename(temp)
    # print(x)
    if panelA is None or panelB is None:
        panelA = Label(image=img)
        panelA.image = img
        panelA.pack(side="left", padx=50, pady=12)
        panelB = Label(image=img)
        panelB.image = img
        panelB.pack(side="right", padx=50, pady=12)
    else:
        panelA.configure(image=img)
        panelB.configure(image=img)
        panelA.image = img
        panelB.image = img

# function defined for cropping image in circle
def crop_fun():
    global  x
    # Open the input image as numpy array, convert to RGB
    img = Image.open(x).convert("RGB")
    npImage = np.array(img)
    h, w = img.size

    # Create same size alpha layer with circle
    alpha = Image.new('L', img.size, 0)
    draw = ImageDraw.Draw(alpha)
    draw.pieslice([0, 0, h, w], 0, 360, fill=255)

    # Convert alpha Image to numpy array
    npAlpha = np.array(alpha)

    # Add alpha layer to RGB
    npImage = np.dstack((npImage, npAlpha))

    # Save with alpha
    Image.fromarray(npImage).save('Cropped_Image.png')

    imge = Image.open('Cropped_Image.png')
    imge = ImageTk.PhotoImage(imge)
    panelB.configure(image=imge)
    panelB.image = imge
    mbox.showinfo("Cropping Status", "Image Cropped in Circle successfully.")

# function defined to reset the edited image to original one
def reset():
    global  x
    # print(x)
    image = cv2.imread(x)[:, :, ::-1]
    global count, eimg
    count = 6
    global o6
    o6 = image
    image = Image.fromarray(o6)
    eimg = image
    image = ImageTk.PhotoImage(image)
    panelB.configure(image=image)
    panelB.image = image
    mbox.showinfo("Success", "Image reset to original format!")

# function defined to same the edited image
def save_img():
    global location, filename, eimg
    print(filename)
    # eimg.save(location + filename + r"_edit.png")
    filename = filedialog.asksaveasfile(mode='w', defaultextension=".jpg")
    if not filename:
        return
    eimg.save(filename)
    mbox.showinfo("Success", "Cropped Image Saved Successfully!")



In [ ]:
# top label
start1 = tk.Label(text = "CIRCULAR IMAGE\nCROPPER", font=("Arial", 50, "underline"), fg="magenta") # same way bg
start1.place(x = 350, y = 10)

# original image label
start1 = tk.Label(text = "Original\nImage", font=("Arial", 40), fg="magenta") # same way bg
start1.place(x = 100, y = 300)

# edited image label
start1 = tk.Label(text = "Circular Cropped\nImage", font=("Arial", 40), fg="magenta") # same way bg
start1.place(x = 800, y = 300)

# save button created
saveb = Button(window, text="SAVE",command=save_img,font=("Arial", 25), bg = "orange", fg = "blue", borderwidth=3, relief="raised")
saveb.place(x =80 , y =20 )

# choose button created
chooseb = Button(window, text="SELECT",command=open_img,font=("Arial", 25), bg = "light green", fg = "blue", borderwidth=3, relief="raised")
chooseb.place(x =170 , y =650 )

# crop button created
enb = Button(window, text="CIRCULAR CROP",command=crop_fun,font=("Arial", 25), bg = "orange", fg = "blue", borderwidth=3, relief="raised")
enb.place(x =530 , y =650 )

# reset button created
resetb = Button(window, text="RESET",command=reset,font=("Arial", 25), bg = "yellow", fg = "blue", borderwidth=3, relief="raised")
resetb.place(x =1000 , y =650 )

# function created for exiting
def exit_win():
    if mbox.askokcancel("Exit", "Do you want to exit?"):
        window.destroy()

# exit button created
exitb = Button(window, text="EXIT",command=exit_win,font=("Arial", 25), bg = "red", fg = "blue", borderwidth=3, relief="raised")
exitb.place(x =1100 , y =20 )


window.protocol("WM_DELETE_WINDOW", exit_win)
window.mainloop()